## This notebook demonstrates bundles to seqex example

Install packages required for apache-beam and protobuf

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from absl import app
from absl import flags
import apache_beam as beam
from google.protobuf import text_format
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2
from proto.stu3 import version_config_pb2
from tensorflow.core.example import example_pb2
from py.google.fhir.seqex import bundle_to_seqex
import time

/usr/local/envs/py2env/lib/python2.7/site-packages/ipykernel/__main__.py:12: ImportWarning: Not importing directory '/usr/local/fhir/proto': missing __init__.py
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/ndimage

In [1]:
def _get_version_config(version_config_path):
  with open(version_config_path) as f:
    return text_format.Parse(f.read(), version_config_pb2.VersionConfig())

# Initialize variables

In [3]:
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

import apache_beam as beam
import re

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'dp-workspace'
google_cloud_options.job_name = 'generate_seqex'
google_cloud_options.staging_location = 'gs://cluster-data/staging'
google_cloud_options.temp_location = 'gs://cluster-data/temp'
options.view_as(StandardOptions).runner = 'DirectRunner'

In [4]:
input_path = 'gs://cluster-data/demo/data/bundles/bundles*'
label_path = 'gs://cluster-data/demo/data/output/labels-00000-of-00001.tfrecords'
output_path = 'gs://cluster-data/demo/data/output/seqex'

## Transform definition

In [5]:
p = beam.Pipeline(options=options)
version_config = _get_version_config("/usr/local/fhir/proto/stu3/version_config.textproto")

keyed_bundles = ( 
    p 
    | 'readBundles' >> beam.io.ReadFromTFRecord(
        input_path, coder=beam.coders.ProtoCoder(resources_pb2.Bundle))
    | 'KeyBundlesByPatientId' >> beam.ParDo(
        bundle_to_seqex.KeyBundleByPatientIdFn()))
event_labels = ( 
    p | 'readEventLabels' >> beam.io.ReadFromTFRecord(
        label_path,
        coder=beam.coders.ProtoCoder(google_extensions_pb2.EventLabel)))
keyed_event_labels = bundle_to_seqex.CreateTriggerLabelsPairLists(
    event_labels)
bundles_and_labels = bundle_to_seqex.CreateBundleAndLabels(
    keyed_bundles, keyed_event_labels)
_ = ( 
    bundles_and_labels
    | 'Reshuffle1' >> beam.Reshuffle()
    | 'GenerateSeqex' >> beam.ParDo(
        bundle_to_seqex.BundleAndLabelsToSeqexDoFn(
            version_config=version_config,
            enable_attribution=False,
            generate_sequence_label=False))
    | 'Reshuffle2' >> beam.Reshuffle()
    | 'WriteSeqex' >> beam.io.WriteToTFRecord(
        output_path,
        coder=beam.coders.ProtoCoder(example_pb2.SequenceExample),
        file_name_suffix='.tfrecords',
        num_shards=2))

/usr/local/envs/py2env/lib/python2.7/site-packages/oauth2client/contrib/gce.py:99: UserWarning: You have requested explicit scopes to be used with a GCE service account.
Using this argument will have no effect on the actual scopes for tokens
requested. These scopes are set at VM instance creation time and
can't be overridden in the request.

  warnings.warn(_SCOPES_WARNING)
I0124 19:58:37.658966 140304687015680 gcsio.py:446] Starting the size estimation of the input
I0124 19:58:37.662934 140304687015680 client.py:614] Attempting refresh to obtain initial access_token
I0124 19:58:37.783057 140304687015680 gcsio.py:460] Finished listing 10 files in 0.12409901619 seconds.
I0124 19:58:37.802076 140304687015680 client.py:614] Attempting refresh to obtain initial access_token
I0124 19:58:37.871395 140304687015680 client.py:614] Attempting refresh to obtain initial access_token


Run the transform

In [ ]:
start = time.time()
p.run().wait_until_finish()
end = time.time()
print(end-start)

I0124 19:58:46.944423 140304687015680 fn_api_runner.py:844] ==================== <function annotate_downstream_side_inputs at 0x7f9ae90785f0> ====================
I0124 19:58:46.947788 140304687015680 fn_api_runner.py:844] ==================== <function fix_side_input_pcoll_coders at 0x7f9ae9078668> ====================
I0124 19:58:46.952614 140304687015680 fn_api_runner.py:844] ==================== <function lift_combiners at 0x7f9ae9078488> ====================
I0124 19:58:46.954586 140304687015680 fn_api_runner.py:844] ==================== <function expand_gbk at 0x7f9ae9078500> ====================
I0124 19:58:46.962486 140304687015680 fn_api_runner.py:844] ==================== <function sink_flattens at 0x7f9ae9078578> ====================
I0124 19:58:46.965171 140304687015680 fn_api_runner.py:844] ==================== <function greedily_fuse at 0x7f9ae90786e0> ====================
I0124 19:58:46.970021 140304687015680 fn_api_runner.py:844] ==================== <function sort_stag

I0124 19:59:36.106637 140304687015680 bundle_processor.py:303] finish <DoOperation KeyEventLabelsByPatientId output_tags=['out'], receivers=[ConsumerSet[KeyEventLabelsByPatientId.out0, coder=WindowedValueCoder[TupleCoder[BytesCoder, LengthPrefixCoder[ProtoCoder]]], len(consumers)=1]]>
I0124 19:59:36.108788 140304687015680 bundle_processor.py:303] finish <DataOutputOperation GroupEventLabelsByPatientId/Write >
I0124 19:59:36.121429 140304687015680 fn_api_runner.py:912] Running (((GroupEventLabelsByPatientId/Read)+(ref_AppliedPTransform_CreateTriggerLabelsPairLists_12))+((ref_AppliedPTransform_GroupBundleAndTriggers/pair_with_trigger_labels_pair_lists_14)+(GroupBundleAndTriggers/Flatten/Transcode/0)))+(GroupBundleAndTriggers/Flatten/Write/0)
I0124 19:59:36.136590 140304687015680 bundle_processor.py:291] start <DataOutputOperation GroupBundleAndTriggers/Flatten/Write/0 >
I0124 19:59:36.138300 140304687015680 bundle_processor.py:291] start <FlattenOperation GroupBundleAndTriggers/Flatten/T

YAY!!